In [4]:
import pandas as pd
import numpy as np
import keras

In [17]:
df = pd.read_fwf('test.txt', header=None)
df.columns=['a','b']
df.drop(['a'], axis=1)

,b
0,Great CD: My lovely Pat has one of the GREAT v...
1,One of the best game music soundtracks - for a...
2,Batteries died within a year ...: I bought thi...
3,"works fine, but Maha Energy is better: Check o..."
4,Great for the non-audiophile: Reviewed quite a...
5,DVD Player crapped out after one year: I also ...
6,"Incorrect Disc: I love the style of this, but ..."
7,DVD menu select problems: I cannot scroll thro...
8,Unique Weird Orientalia from the 1930's: Exoti...
9,"Not an ""ultimate guide"": Firstly,I enjoyed the..."


In [20]:
import csv
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/0x23/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/0x23/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
text_lines = list()
lines = df['b'].values.tolist()

for line in lines:
    tokens = word_tokenize(line)
    
    # tokens = [w.lower() for w in tokens]  # convert to lower case

    table = str.maketrans('', '', string.punctuation) # remove punctuation
    stripped = [w.translate(table) for w in tokens] 

    words = [word for word in stripped if word.isalpha()] # remove remaining non alphabetic tokens

    stop_words = set(stopwords.words('english')) # remove english stopwords
    words = [w for w in words if not w in stop_words]
    text_lines.append(words)

In [22]:
print(len(text_lines))

400000


In [23]:
import gensim
vocabulary = gensim.models.Word2Vec(sentences=text_lines, size=200, window=5, workers=4, min_count=1)
words = list(vocabulary.wv.vocab)
print('Vocabulary size:  %d'  % len(words))

Vocabulary size:  470787


In [24]:
vocabulary.wv.most_similar('see')

[('hear', 0.5714476108551025),
 ('find', 0.520993173122406),
 ('tell', 0.5195527672767639),
 ('know', 0.5086527466773987),
 ('watch', 0.5077857971191406),
 ('discover', 0.5029417872428894),
 ('figure', 0.5005062222480774),
 ('imagine', 0.49845755100250244),
 ('notice', 0.49817609786987305),
 ('seeing', 0.4932604134082794)]

In [25]:
filename = 'embeddings.txt'
vocabulary.wv.save_word2vec_format(filename, binary=False)

In [1]:
from gensim.models import KeyedVectors

# Load gensim word2vec
w2v_path = 'embeddings.txt'
w2v = KeyedVectors.load_word2vec_format(w2v_path)

import io

# Vector file, `\t` seperated the vectors and `\n` seperate the words
"""
0.1\t0.2\t0.5\t0.9
0.2\t0.1\t5.0\t0.2
0.4\t0.1\t7.0\t0.8
"""
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')

# Meta data file, `\n` seperated word
"""
token1
token2
token3
"""
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# Write meta file and vector file
for index in range(len(w2v.index2word)):
    word = w2v.index2word[index]
    vec = w2v.vectors[index]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

# Then we can visuale using the `http://projector.tensorflow.org/` to visualize those two files.

# 1. Open the Embedding Projector.
# 2. Click on "Load data".
# 3. Upload the two files we created above: vecs.tsv and meta.tsv.